# Import stuff

In [1]:
%matplotlib inline
%matplotlib notebook

import sys
import PIL
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits

# Load the data

In [2]:
f = fits.open('/home/sean/Downloads/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fixed.fits') # LoTSS
# '/mnt/closet/data/lotss.fits', 'LOFAR_HBA_T1_DR1_merge_ID_optical_v0.7.fits'

h = f[1]
lotss_ra  = h.data['RA' ] # degrees
lotss_dec = h.data['DEC'] # degrees
f.close()

f = fits.open('/mnt/closet/data/3fgl.fits') # 3FGL
h = f[1]
fgl_ra  = h.data['RAJ2000']
fgl_dec = h.data['DEJ2000']
fgl_l   = h.data['GLON   ']
fgl_b   = h.data['GLAT   ']
f.close()

f = np.genfromtxt('/mnt/closet/data/bzcat.txt', skip_header = 1, delimiter = '|', dtype = str) # BZCAT
bz_ra  = f[:, 2].astype(np.float)
bz_dec = f[:, 3].astype(np.float)

f = np.genfromtxt('/mnt/closet/data/tevcat.txt', delimiter = '\t', dtype = float, usecols = (0, 1)) # TEVCAT
tev_ra  = f[:, 0]
tev_dec = f[:, 1]

# Convert to galactic coordinates

In [3]:
def convert(ra, dec):
    c = SkyCoord(ra = ra * u.degree, dec = dec * u.degree, frame = 'icrs')
    l = c.galactic.l
    b = c.galactic.b
    return l, b

lotss_l, lotss_b = convert(lotss_ra, lotss_dec)
bz_l, bz_b = convert(bz_ra, bz_dec)
tev_l, tev_b = convert(tev_ra, tev_dec)

# Function to combine images

In [4]:
def combine(file1, file2):
    
    list_im   = [file1, file2]
    imgs      = [PIL.Image.open(i) for i in list_im]
    min_shape = sorted([(np.sum(i.size), i.size) for i in imgs])[0][1]
    imgs_comb = np.hstack((np.asarray(i.resize(min_shape)) for i in imgs))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    imgs_comb.save('sky-map-combined.png') 

# Function to plot the data

In [5]:
def plotting(lotss1, lotss2, bz1, bz2, fgl1, fgl2, tev1, tev2, coordinates, plot, size):

    color = 'black'
    if plot:
        color = 'white'
    if size == 'all':
        markers = 4
    else:
        markers = 6

    plt.rcParams['font.family'] = 'serif'
    plt.figure(figsize = (13.92, 8.60))
    
    plt.plot(lotss1, lotss2, ls = 'None', color = 'tomato', marker = '.', markersize = 4, label = 'LoTSS')
    plt.plot(fgl1, fgl2, ls = 'None', color = 'darkseagreen', marker = 's', markersize = markers, label = '3FGL')
    plt.plot(bz1, bz2, ls = 'None', color = 'royalblue', marker = '^', markersize = markers, label = 'BZCAT')
    plt.plot(tev1, tev2, ls = 'None', color = 'palevioletred', marker = '8', markersize = markers, label = 'TEVCAT')
        
    if size == 'all':
        if coordinates == 'ra, dec':
            plt.xlim(0, 360)
            plt.ylim(-90, 90)
            plt.xticks(range(0, 361, 90), fontsize = 30, color = color)
            plt.yticks(range(-45, 91, 45), fontsize = 30, color = color)            
            
        else:
            plt.xlim(0, 180)
            plt.ylim(-90, 90)
            plt.xticks(range(0, 361, 90), [-180, -90, 0, 90, 180], fontsize = 30, color = color)
            plt.yticks(range(-45, 91, 45), fontsize = 30, color = color)
        
    else:
        if coordinates == 'ra, dec':
            plt.xlim(min(lotss_ra), max(lotss_ra))
            plt.ylim(min(lotss_dec), max(lotss_dec))
            plt.xticks([180], fontsize = 30, color = color)
            plt.yticks([], [], fontsize = 30, color = color)            
                       
        else:
            plt.xlim(76.322, 166.807)
            plt.ylim(51.4536, 71.2585)
            plt.xticks(range(90, 166, 90), fontsize = 30, color = color)
            plt.yticks([], [], fontsize = 30, color = color)
        
    if coordinates == 'ra, dec':
        plt.xlabel('Right ascension (degrees)', fontsize = 30, color = color)
        plt.ylabel('Declination (degrees)', fontsize = 30, color = color)
        
    else:
        plt.xlabel('Longitude (degrees)', fontsize = 30, color = color)
        plt.ylabel('Latitude (degrees)', fontsize = 30, color = color)
   
    plt.grid(color = 'black', linestyle = 'dashed', linewidth = 1)
    
    legend = plt.legend(bbox_to_anchor = (0, 1.0, 1, 0), loc = 'lower left', mode = 'expand', numpoints = 1, fontsize = 30, ncol = 4,
                     frameon = False, markerscale = 5)
    plt.setp(legend.get_texts(), color = color)

    if not plot:
        plt.savefig('sky-map-' + str(size) + '.png', bbox_inches = 'tight', format = 'png')
    elif plot:
        plt.show()

# Make the pliot

In [ ]:
make_plots = 'ra, dec'
plot = False

if make_plots == 'ra, dec':
    plotting(lotss_ra, lotss_dec, bz_ra, bz_dec, fgl_ra, fgl_dec, tev_ra, tev_dec, 'ra, dec', plot, 'all')
    plotting(lotss_ra, lotss_dec, bz_ra, bz_dec, fgl_ra, fgl_dec, tev_ra, tev_dec, 'ra, dec', plot, 'lotss')
else:
    plotting(lotss_l, lotss_b, bz_l, bz_b, fgl_l, fgl_b, tev_l, tev_b, 'l, b', plot, 'all')    
    plotting(lotss_l, lotss_b, bz_l, bz_b, fgl_l, fgl_b, tev_l, tev_b, 'l, b', plot, 'lotss')
    
if not plot:
    combine('sky-map-all.png', 'sky-map-lotss.png')